In [ ]:
import torch
from pathlib import Path
from sklearn.metrics import classification_report
from tqdm import tqdm

# Define paths
model_path = "labeledyolov5.pt"  # Path to the trained YOLOv5 model
test_images_path = Path("../Data/medical_business_data/test/images")  # Test images folder
test_labels_path = Path("../Data/medical_business_data/test/labels")  # Test labels folder

# Load the YOLOv5 model
model = torch.hub.load('../yolov5', 
                       'custom', path=model_path, source='local')

# Function to parse YOLO format labels
def parse_yolo_labels(label_path):
    labels = []
    if not label_path.exists():  # Check if label file exists
        return labels
    with open(label_path, 'r') as file:
        for line in file.readlines():
            data = line.split()
            class_id = int(data[0])  # Class label
            labels.append(class_id)
    return labels

# Prepare ground truth and predictions
y_true = []  # Ground truth labels
y_pred = []  # Predicted labels

# Iterate through test images
image_files = sorted(test_images_path.glob("*.jpg"))  
for image_path in tqdm(image_files, desc="Evaluating"):
    # Inference
    results = model(image_path)
    predictions = results.xyxy[0]  # Bounding box predictions [x1, y1, x2, y2, conf, class]

    # Extract predicted class IDs
    pred_classes = predictions[:, -1].tolist() if len(predictions) > 0 else []

    # Get ground truth labels for the current image
    label_file = test_labels_path / (image_path.stem + ".txt")  # YOLO format labels
    ground_truth_classes = parse_yolo_labels(label_file)

    # Match predictions and ground truth sizes
    max_len = max(len(ground_truth_classes), len(pred_classes))
    ground_truth_classes.extend([-1] * (max_len - len(ground_truth_classes)))  # Pad missing ground truth with -1
    pred_classes.extend([-1] * (max_len - len(pred_classes)))  # Pad missing predictions with -1

    # Add to lists
    y_true.extend(ground_truth_classes)
    y_pred.extend(pred_classes)

# Extract class names from the model
class_names = list(model.names.values())  # Convert dict values to a list
num_classes = len(class_names)

# Generate evaluation metrics
print("Classification Report:")
print(classification_report(
    y_true,
    y_pred,
    labels=range(num_classes),  # Only evaluate valid classes
    target_names=class_names    # Use class names for the report
))


YOLOv5  v7.0-374-g94a62456 Python-3.9.20 torch-2.5.0 CUDA:0 (NVIDIA GeForce RTX 4050 Laptop GPU, 6140MiB)

Fusing layers... 
Model summary: 157 layers, 7029004 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
Evaluating:   0%|          | 0/24 [00:00<?, ?it/s]C:\Users/elbet/OneDrive/Desktop/Ten/week-7/github/Building-a-data-warehouse/yolov5\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users/elbet/OneDrive/Desktop/Ten/week-7/github/Building-a-data-warehouse/yolov5\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
C:\Users/elbet/OneDrive/Desktop/Ten/week-7/github/Building-a-data-warehouse/yolov5\models\common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args.

Classification Report:
                  precision    recall  f1-score   support

        Brochure       0.50      0.80      0.62        10
Cosmetic Product       0.00      0.00      0.00         6
 Medicine Bottle       0.00      0.00      0.00         9
     Stethoscope       0.00      0.00      0.00         1
         Syringe       0.00      0.00      0.00         0
  Tablet Package       0.00      0.00      0.00         3
            food       0.00      0.00      0.00         5

       micro avg       0.47      0.24      0.31        34
       macro avg       0.07      0.11      0.09        34
    weighted avg       0.15      0.24      0.18        34

